# Analyze data using ilastik
The notebook shows how load image contained in a dataset
from OMERO and analyze them in ilastik.
It assumes that ilastik project is linked to the dataset.
The images are taken from  the paper "NesSys: a novel method for accurate nuclear segmentation in 3D" published August 2019 in PLOS Biology: https://doi.org/10.1371/journal.pbio.3000388

The images can be viewed online in the [Image Data Resource](https://idr.openmicroscopy.org/webclient/?show=project-801).

The metadata are read from OMERO and the Binary data are read from S3.
The ilastik results are saved as a zarr file and saved back to OMERO.
The order might need to be adjusted depending on the ilastik project.

### Insert required packages

In [1]:
import numpy
import os
import itertools
import zarr
import dask.array as da

import tempfile

import omero.clients
from omero.gateway import BlitzGateway
from getpass import getpass

from ilastik import app
from ilastik.applets.dataSelection.opDataSelection import PreloadedArrayDatasetInfo
import vigra
import ipywidgets as widgets
# package for 3d visualization
from itkwidgets import view

/srv/conda/envs/notebook/lib/python3.7/site-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 7.1.0. Several security issues (CVE-2020-11538, CVE-2020-10379, CVE-2020-10994, CVE-2020-10177) have been fixed in pillow 7.1.0 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


### Create a connection to an OMERO server

In [2]:
HOST = 'wss://outreach.openmicroscopy.org/omero-ws'
conn = BlitzGateway(input("Username: "),
                    getpass("OMERO Password: "),
                    host=HOST, secure=True)
print(conn.connect())
conn.c.enableKeepAlive(60)

Username: trainer-1
OMERO Password: ········
True


### Enter the dataset ID

In [3]:
dataset_id = 6161

### Load the ilastik projects linked to the dataset

In [4]:
def load_model(dataset_id, path):
    dataset = conn.getObject("Dataset", dataset_id)
    # Go through all the annotations on the Dataset
    options = []
    for ann in dataset.listAnnotations():
        if isinstance(ann, omero.gateway.FileAnnotationWrapper):
            name = ann.getFile().getName()
            # Select the ilatisk project TODO: use namespace
            if name.endswith(".ilp"):
                file_path = os.path.join(path, name)
                options.append((name, file_path))
                with open(str(file_path), 'wb') as f:
                    for chunk in ann.getFileInChunks():
                        f.write(chunk)
    return widgets.Dropdown(options=options, disabled=False)

### Load an Image as 5D-numpy array: order TZYXC

In [5]:
def load_from_s3(image, resolution='0'):
    id = image.getId()
    endpoint_url = 'https://minio-dev.openmicroscopy.org/'
    root = 'idr/outreach/%s.zarr/' % id
    # data.shape is (t, c, z, y, x) by convention
    data = da.from_zarr(endpoint_url + root)
    values = data[:]
    values = values.swapaxes(1, 2).swapaxes(2, 3).swapaxes(3, 4)
    return numpy.asarray(values)

### Create a temporary directory where to download the ilastik project

In [6]:
path = tempfile.mkdtemp()
if not os.path.exists(path):
    os.makedirs(path)

### Select the ilastik project to use.

In [7]:
model_selection = load_model(dataset_id, path)
display(model_selection)

Dropdown(options=(('pixel-class-133.ilp', '/tmp/tmpn4hz7v8z/pixel-class-133.ilp'),), value='/tmp/tmpn4hz7v8z/p…

### Load each image as a 5D-numpy array and analyze.

In [ ]:
# Load the model linked to the dataset
model_file = model_selection.value


images = conn.getObjects('Image', opts={'dataset': dataset_id})

# Prepare ilastik
os.environ["LAZYFLOW_THREADS"] = "2"
os.environ["LAZYFLOW_TOTAL_RAM_MB"] = "2000"
args = app.parse_args([])
args.headless = True
args.project = model_file
shell = app.main(args)

values = itertools.islice(images, 2)
for image in values:
    filename, file_extension = os.path.splitext(image.getName())
    input_data = load_from_s3(image)

    # run ilastik headless
    print('running ilastik using %s and %s' % (model_file, image.getName()))
    role_data_dict = [ {"Raw Data": PreloadedArrayDatasetInfo(preloaded_array=input_data, axistags=vigra.defaultAxistags("tzyxc"))}]
    predictions = shell.workflow.batchProcessingApplet.run_export(role_data_dict, export_to_array=True)
    for data in predictions:
        # Re-organise array from tzyxc to zctyx order expected by OMERO
        data = data.swapaxes(0, 1).swapaxes(3, 4).swapaxes(2, 3).swapaxes(1, 2)
        data_viewer = data[:, 0, 0, :, :]
print("done")

INFO ilastik.app: config file location: <none>


INFO:ilastik.app:config file location: <none>


INFO ilastik.app: Starting ilastik from "/srv/conda/envs/notebook/ilastik-meta".


INFO:ilastik.app:Starting ilastik from "/srv/conda/envs/notebook/ilastik-meta".


Starting ilastik from "/srv/conda/envs/notebook/ilastik-meta".
INFO ilastik.app: Resetting lazyflow thread pool with 2 threads.


...

DEBUG:lazyflow.operators.classifierOperators:Features took 7.112726 seconds. Prediction took 2.729605 seconds. Subregion: start '[0, 92, 184, 0]' stop '[92, 184, 276, 2]'


DEBUG lazyflow.operators.classifierOperators: Features took 6.069309 seconds. Prediction took 2.700557 seconds. Subregion: start '[0, 92, 276, 0]' stop '[92, 184, 325, 2]'


DEBUG:lazyflow.operators.classifierOperators:Features took 6.069309 seconds. Prediction took 2.700557 seconds. Subregion: start '[0, 92, 276, 0]' stop '[92, 184, 325, 2]'


DEBUG lazyflow.operators.classifierOperators: Features took 8.147896 seconds. Prediction took 2.632021 seconds. Subregion: start '[0, 184, 0, 0]' stop '[92, 276, 92, 2]'


DEBUG:lazyflow.operators.classifierOperators:Features took 8.147896 seconds. Prediction took 2.632021 seconds. Subregion: start '[0, 184, 0, 0]' stop '[92, 276, 92, 2]'


### View the results.

In [9]:
view(data_viewer)

Viewer(rendered_image=<itkImagePython.itkImageF3; proxy of <Swig Object of type 'itkImageF3 *' at 0x7f0354361f…

### Close the connection to the OMERO server

In [10]:
conn.close()

# License
Copyright (C) 2019-2021 University of Dundee. All Rights Reserved.
This program is free software; you can redistribute it and/or modify it
under the terms of the GNU General Public License as published by the
Free Software Foundation; either version 2 of the License, or
(at your option) any later version.
This program is distributed in the hope that it will be useful, but
WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY
or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for
more details. You should have received a copy of the GNU General
Public License along with this program; if not, write to the
Free Software Foundation,
Inc., 51 Franklin Street, Fifth Floor, Boston, MA 02110-1301 USA.